In [1]:
from datasets import load_dataset

dataset = load_dataset("imdb")

/home/rafael/projects/bullet/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Zero Shot Learning

In [2]:
from bullet.core.sentiment import SentimentClassifier

df_test_sample = dataset["test"].to_pandas().sample(n = 100)

classifier = SentimentClassifier()
result = classifier.predict_pandas(df_test_sample)

In [4]:
result

ClassificationResponse(results=[PromptResponse(response='\nNEG', embeddings_for_model='gpt-3.5-turbo-instruct', encoding=None, label=0), PromptResponse(response='\nNEG', embeddings_for_model='gpt-3.5-turbo-instruct', encoding=None, label=0), PromptResponse(response='\nNEG', embeddings_for_model='gpt-3.5-turbo-instruct', encoding=None, label=0), PromptResponse(response='\nNEG', embeddings_for_model='gpt-3.5-turbo-instruct', encoding=None, label=0), PromptResponse(response='\nNEG', embeddings_for_model='gpt-3.5-turbo-instruct', encoding=None, label=0), PromptResponse(response='\nPOS', embeddings_for_model='gpt-3.5-turbo-instruct', encoding=None, label=1), PromptResponse(response='\nNEG', embeddings_for_model='gpt-3.5-turbo-instruct', encoding=None, label=0), PromptResponse(response='\nPOS', embeddings_for_model='gpt-3.5-turbo-instruct', encoding=None, label=1), PromptResponse(response='\nNEG', embeddings_for_model='gpt-3.5-turbo-instruct', encoding=None, label=0), PromptResponse(response

In [5]:
df_test_sample["predicted"] = [response.label for response in result.results]

In [6]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
from sklearn.metrics import classification_report

report = classification_report(df_test_sample.label, df_test_sample.predicted)
print(report)

              precision    recall  f1-score   support

           0       0.95      0.91      0.93        57
           1       0.89      0.93      0.91        43

    accuracy                           0.92       100
   macro avg       0.92      0.92      0.92       100
weighted avg       0.92      0.92      0.92       100



## Few-Shot Learning

### Number of examples = 3

In [14]:
# Define Few Shot examples

df_train_sample = dataset["train"].to_pandas().sample(3, random_state = 123)

template = "Review: \"{review}\"\nLabel: \"{label}\""
examples = [
    template.format(
        review = row["text"],
        label = "POS" if row["label"] == 1 else "NEG"
    )
    for _, row
    in df_train_sample.iterrows()
]

examples

['Review: "I think this still is the best routine. There are some others, like Rock\'s "bring the pain", and Allen\'s "Men are Pigs" that are hilarious; "Damon Waynes last stand" is also funny in a tearful way - but this routine has no errors. All the jokes are funny, and the time limit of 70 minutes is perfect. Just long enough to last 20 years. I just love how he allows the audience to be totally themselves and unrestricted. I\'m a fan of the classics and for a guy who watched a lot of of Jim Carrey growing up, watching a more laid back comic is pretty cool. Not putting in a category with Ellen and Newhart, but something you can watch if you\'re bloated. Thanks Eddie, god bless."\nLabel: "POS"',
 'Review: "It is no wonder this movie won 4 prices, it is a movie that lingers to any soul, it isn\'t a wonder why it took Paul Reiser 20 years to finally give in and talk to Peter Falk about his idea. I can understand every part of it, this is a movie that will make you cry just a tear, or t

In [15]:
reviews = df_test_sample.text.values

results = classifier.predict_few_shot(
    reviews = reviews,
    examples = examples
)

In [16]:
df_test_sample["predicted"] = [response.label for response in results.results]

report = classification_report(df_test_sample.label, df_test_sample.predicted)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.95      0.94        57
           1       0.93      0.91      0.92        43

    accuracy                           0.93       100
   macro avg       0.93      0.93      0.93       100
weighted avg       0.93      0.93      0.93       100



### Number of examples = 5

In [ ]:
# Define Few Shot examples

df_train_sample = dataset["train"].to_pandas().sample(5, random_state = 123)

template = "Review: \"{review}\"\nLabel: \"{label}\""
examples = [
    template.format(
        review = row["text"],
        label = "POS" if row["label"] == 1 else "NEG"
    )
    for _, row
    in df_train_sample.iterrows()
]

examples

In [ ]:
results = classifier.predict_few_shot(
    reviews = reviews,
    examples = examples
)

In [ ]:
df_test_sample["predicted"] = [response.label for response in results.results]

report = classification_report(df_test_sample.label, df_test_sample.predicted)
print(report)